<a href="https://colab.research.google.com/github/JJEEEFFFF/CE-888-Assignment-2/blob/main/CE_888_Assessment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CE-888 Assignment-2 seq to SQL **
Importing the required Libraries 

In [ ]:
import re
import json
import pathlib
import random
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

Importing the Dataset in colab

In [ ]:
from google.colab import files
Spider_dataset_1=files.upload()
Spider_dataset_2=files.upload()

Saving train_others.json to train_others.json


Saving train_spider.json to train_spider.json



Retrieveing the Datasets

In [ ]:
train_path = '/content/train_others.json'
train_other_path = '/content/train_spider.json'


train_data = pd.read_json(train_path)
train_other_data = pd.read_json(train_other_path)

train_data = train_data.head(1000)
train_other_data = train_other_data.head(1000)

**Concatenate Datasets**

In [ ]:
train_data = pd.concat([train_data, train_other_data], axis=0, ignore_index=True)
train_data.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",what is the biggest city in wyoming,"[what, is, the, biggest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], ..."
1,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",what wyoming city has the largest population,"[what, wyoming, city, has, the, largest, popul...","{'from': {'table_units': [['table_unit', 1]], ..."
2,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",what is the largest city in wyoming,"[what, is, the, largest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], ..."
3,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",where is the most populated area of wyoming,"[where, is, the, most, populated, area, of, wy...","{'from': {'table_units': [['table_unit', 1]], ..."
4,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",which city in wyoming has the largest population,"[which, city, in, wyoming, has, the, largest, ...","{'from': {'table_units': [['table_unit', 1]], ..."


Creating  the List of 'Query' and 'Question' values from the dataset

In [ ]:
text_pairs = []
for i in range (len(train_data)):
    question=train_data.loc[i, "question"]
    query=train_data.loc[i, "query"]
    query = "[start] " + query + " [end]"
    text_pairs.append((question, query))

Creating the Training, Testing & Validation Sets


In [ ]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

In [ ]:
print ('Total Training Pairs:', len(train_pairs))
print ('Total Testing Pairs:', len(test_pairs))
print ('Total Validation Pairs:', len(val_pairs))

Total Training Pairs: 1400
Total Testing Pairs: 300
Total Validation Pairs: 300


Performing Data Preprocessing that involves casefolding and Text Vectorizsation

In [ ]:
vocab_size = 15000
sequence_length = 64
batch_size = 128

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return lowercase

question_vectorization = TextVectorization(max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,)
query_vectorization = TextVectorization(max_tokens=vocab_size,  output_mode="int", output_sequence_length=sequence_length + 1, standardize=custom_standardization)


In [ ]:
train_question_texts = [pair[0] for pair in train_pairs]
train_query_texts = [pair[1] for pair in train_pairs]
question_vectorization.adapt(train_question_texts)
query_vectorization.adapt(train_query_texts)

In [ ]:
def format_dataset(question, query):
    question = question_vectorization(question)
    query = query_vectorization(query)
    return ({"encoder_inputs": question, "decoder_inputs": query[:, :-1],}, query[:, 1:])

def make_dataset(pairs):
    question_texts, query_texts = zip(*pairs)
    question_texts = list(question_texts)
    query_texts = list(query_texts)
    dataset = tf.data.Dataset.from_tensor_slices((question_texts, query_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

In [ ]:
train_ds = make_dataset(train_pairs)
test_ds = make_dataset(test_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (128, 64)
inputs["decoder_inputs"].shape: (128, 64)
targets.shape: (128, 64)


# Setting Parameters for the Embeddings

In [ ]:
embed_dim = 256
latent_dim = 1024

## Model 1: Single Layer Bidirectional GRU

In [ ]:
source = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(layers.GRU(latent_dim), merge_mode="sum")(x)

In [ ]:
past_target = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)

In [ ]:
seq2seq_01 = keras.Model([source, past_target], target_next_step)

## Setup Callbacks

In [ ]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=1),
             ModelCheckpoint("seq2seq_01.keras", save_best_only=True, monitor="val_accuracy", mode='max')]

## Compile and Run the model

In [ ]:
seq2seq_01.compile(
optimizer="rmsprop",
loss="sparse_categorical_crossentropy",
metrics=["accuracy"])

seq2seq_01.fit(train_ds, epochs=30, validation_data=val_ds, callbacks=callbacks)

Epoch 1/30
11/11 [==============================] - 353s 31s/step - loss: 2.4581 - accuracy: 0.1006 - val_loss: 1.7839 - val_accuracy: 0.1713
Epoch 2/30
11/11 [==============================] - 337s 31s/step - loss: 1.6564 - accuracy: 0.1519 - val_loss: 1.8059 - val_accuracy: 0.2192
Epoch 3/30
11/11 [==============================] - 336s 31s/step - loss: 1.5252 - accuracy: 0.2030 - val_loss: 1.7847 - val_accuracy: 0.2603
Epoch 4/30
11/11 [==============================] - 347s 32s/step - loss: 1.3644 - accuracy: 0.2857 - val_loss: 1.7427 - val_accuracy: 0.3048
Epoch 5/30
11/11 [==============================] - 336s 31s/step - loss: 1.1273 - accuracy: 0.3973 - val_loss: 1.6861 - val_accuracy: 0.3398
Epoch 6/30
11/11 [==============================] - 339s 31s/step - loss: 0.9484 - accuracy: 0.4707 - val_loss: 1.5955 - val_accuracy: 0.3770
Epoch 7/30
11/11 [==============================] - 332s 30s/step - loss: 0.8163 - accuracy: 0.5242 - val_loss: 1.6079 - val_accuracy: 0.3945
Epoch 

In [ ]:
query_vocab = query_vectorization.get_vocabulary()
query_index_lookup = dict(zip(range(len(query_vocab)), query_vocab))
max_decoded_sentence_length = 20

In [ ]:
def decode_sequence_01(input_sentence):
  tokenized_input_sentence = question_vectorization([input_sentence])
  decoded_sentence = "[start]"
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = query_vectorization([decoded_sentence])
    next_token_predictions = seq2seq_01.predict([tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(next_token_predictions[0, i, :])

    sampled_token = query_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence

In [ ]:
actual, predicted = list(), list()
question_texts = [pair[0] for pair in test_pairs]
query_texts = [pair[1] for pair in test_pairs]
for i in range(len(question_texts)):
  input_sentence = question_texts[i]
  output_sentence = query_texts[i]
  y_pred= decode_sequence_01(input_sentence)
  actual.append(output_sentence)
  predicted.append(y_pred)
    
# calcuate BLEU score
print("BLEU-1 Score on Test Dataset: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))

BLEU-1 Score on Test Dataset: 0.121104


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
test_questions_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
  input_sentence = random.choice(test_questions_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence_01(input_sentence))

-
What are distinct locations where tracks are located?
[start] select state_name from river where state_name in ( select border from river where state_name in ( select border from border_info
-
What is the transaction type that has processed the greatest total amount in transactions?
[start] select state_name from river where state_name in ( select border from river where state_name in ( select border from river
-
Find the policy type used by more than 4 customers.
[start] select state_name from river where state_name in ( select border from river where state_name in ( select border from border_info
-
Show the enrollment and primary_conference of the oldest college.
[start] select state_name from river where state_name in ( select border from river where state_name in ( select border from river
-
What is the product, chromosome, and porphyria of the enzymes located at 'Cytosol'?
[start] select state_name from river where state_name in ( select border from river where state_name in ( s

# Compute BLEU Score on Validation Dataset

In [ ]:
actual, predicted = list(), list()
question_texts = [pair[0] for pair in val_pairs]
query_texts = [pair[1] for pair in val_pairs]
for i in range(len(question_texts)):
  input_sentence = question_texts[i]
  output_sentence = query_texts[i]
  y_pred= decode_sequence_01(input_sentence)
  actual.append(output_sentence)
  predicted.append(y_pred)
    
# calcuate BLEU score
print("BLEU-1 Score on Validation Dataset: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))

BLEU-1 Score on Validation Dataset: 0.125224


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
source = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
out_encoder, state_h_forward, state_c_forward, state_h_backward, state_c_backward = layers.Bidirectional(layers.LSTM(latent_dim, return_sequences=True, return_state=True), merge_mode="sum")(x)


In [ ]:
past_target = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_LSTM = layers.LSTM(latent_dim, return_sequences=True)

encoder_state=[state_h_forward, state_c_forward]

x = decoder_LSTM(x, initial_state=encoder_state)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)

In [ ]:
seq2seq_02 = keras.Model([source, past_target], target_next_step)

In [ ]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=1),
             ModelCheckpoint("seq2seq_02.keras", save_best_only=True, monitor="val_accuracy", mode='max')]


In [ ]:
seq2seq_02.compile(
optimizer="rmsprop",
loss="sparse_categorical_crossentropy",
metrics=["accuracy"])

seq2seq_02.fit(train_ds, epochs=30, validation_data=val_ds, callbacks=callbacks)

Epoch 1/30
11/11 [==============================] - 341s 30s/step - loss: 2.3651 - accuracy: 0.1072 - val_loss: 1.7924 - val_accuracy: 0.1049
Epoch 2/30
11/11 [==============================] - 330s 30s/step - loss: 1.6775 - accuracy: 0.1312 - val_loss: 1.7903 - val_accuracy: 0.1434
Epoch 3/30
11/11 [==============================] - 325s 30s/step - loss: 1.5930 - accuracy: 0.1477 - val_loss: 1.7987 - val_accuracy: 0.1621
Epoch 4/30
11/11 [==============================] - 327s 30s/step - loss: 1.5334 - accuracy: 0.1666 - val_loss: 1.7634 - val_accuracy: 0.1881
Epoch 5/30
11/11 [==============================] - 325s 30s/step - loss: 1.4818 - accuracy: 0.1903 - val_loss: 1.6838 - val_accuracy: 0.2172
Epoch 6/30
11/11 [==============================] - 325s 30s/step - loss: 1.3706 - accuracy: 0.2214 - val_loss: 1.6653 - val_accuracy: 0.2238
Epoch 7/30
11/11 [==============================] - 320s 29s/step - loss: 1.2704 - accuracy: 0.2552 - val_loss: 1.6490 - val_accuracy: 0.2487
Epoch 

In [ ]:
query_vocab = query_vectorization.get_vocabulary()
query_index_lookup = dict(zip(range(len(query_vocab)), query_vocab))
max_decoded_sentence_length = 20

In [ ]:
def decode_sequence_02(input_sentence):
  tokenized_input_sentence = question_vectorization([input_sentence])
  decoded_sentence = "[start]"
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = query_vectorization([decoded_sentence])
    next_token_predictions = seq2seq_02.predict([tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(next_token_predictions[0, i, :])

    sampled_token = query_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence

In [ ]:
actual, predicted = list(), list()
question_texts = [pair[0] for pair in test_pairs]
query_texts = [pair[1] for pair in test_pairs]
for i in range(len(question_texts)):
  input_sentence = question_texts[i]
  output_sentence = query_texts[i]
  y_pred= decode_sequence_02(input_sentence)
  actual.append(output_sentence)
  predicted.append(y_pred)
    
# calcuate BLEU score
print("BLEU-1 Score on Test Dataset: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))

BLEU-1 Score on Test Dataset: 0.136778


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
test_questions_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
  input_sentence = random.choice(test_questions_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence_02(input_sentence))

-
Show the shop addresses ordered by their opening year.
[start] select state_name from state where state_name = ( select min ( area ) from state ); [end]
-
Please list the name and id of all artists that have at least 3 albums in alphabetical order.
[start] select state_name from state where state_name = ( select max ( area ) from state where state_name = ( select
-
What are the invoice dates for customers with the first name Astrid and the last name Gruber?
[start] select state_name from state where state_name = ( select max ( area ) from state where state_name = ( select
-
What are the duration of the longest and shortest pop tracks in milliseconds?
[start] select state_name from state where state_name = ( select min ( area ) from state where state_name = ( select
-
Find the names of artists that do not have any albums.
[start] select state_name from state where state_name = ( select min ( area ) from state where state_name = ( select


In [ ]:
actual, predicted = list(), list()
question_texts = [pair[0] for pair in val_pairs]
query_texts = [pair[1] for pair in val_pairs]
for i in range(len(question_texts)):
  input_sentence = question_texts[i]
  output_sentence = query_texts[i]
  y_pred= decode_sequence_02(input_sentence)
  actual.append(output_sentence)
  predicted.append(y_pred)
    
# calcuate BLEU score
print("BLEU-1 Score on Validation Dataset: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))

BLEU-1 Score on Validation Dataset: 0.144720


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


# Model 3: Two Layer Bidirectional LSTM

In [ ]:
source = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(layers.LSTM(latent_dim), merge_mode="sum")(x)
out_encoder1, state_h_forward1, state_c_forward1, state_h_backward1, state_c_backward1 = layers.Bidirectional(layers.LSTM(latent_dim, return_sequences=True, return_state=True), merge_mode="sum")(x)
out_encoder2, state_h_forward2, state_c_forward2, state_h_backward2, state_c_backward2 = layers.Bidirectional(layers.LSTM(latent_dim, return_sequences=True, return_state=True), merge_mode="sum")(out_encoder1)

In [ ]:
past_target = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_LSTM = layers.LSTM(latent_dim, return_sequences=True)

encoder_state=[state_h_forward2, state_c_forward2]

x = decoder_LSTM(x, initial_state=encoder_state)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)

In [ ]:
seq2seq_03 = keras.Model([source, past_target], target_next_step)

In [ ]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=1),
             ModelCheckpoint("seq2seq_03.keras", save_best_only=True, monitor="val_accuracy", mode='max')]

In [ ]:
seq2seq_03.compile(
optimizer="rmsprop",
loss="sparse_categorical_crossentropy",
metrics=["accuracy"])

seq2seq_03.fit(train_ds, epochs=30, validation_data=val_ds, callbacks=callbacks)

Epoch 1/30
11/11 [==============================] - 547s 48s/step - loss: 2.3570 - accuracy: 0.1023 - val_loss: 1.7486 - val_accuracy: 0.1444
Epoch 2/30
11/11 [==============================] - 518s 47s/step - loss: 1.6675 - accuracy: 0.1377 - val_loss: 1.7716 - val_accuracy: 0.1514
Epoch 3/30
11/11 [==============================] - 525s 48s/step - loss: 1.5876 - accuracy: 0.1511 - val_loss: 1.7713 - val_accuracy: 0.1700
Epoch 4/30
11/11 [==============================] - 525s 48s/step - loss: 1.5353 - accuracy: 0.1716 - val_loss: 1.7373 - val_accuracy: 0.1940
Epoch 5/30
11/11 [==============================] - 524s 48s/step - loss: 1.4495 - accuracy: 0.1989 - val_loss: 1.6811 - val_accuracy: 0.2247
Epoch 6/30
11/11 [==============================] - 527s 48s/step - loss: 1.3515 - accuracy: 0.2276 - val_loss: 1.6664 - val_accuracy: 0.2176


In [ ]:
query_vocab = query_vectorization.get_vocabulary()
query_index_lookup = dict(zip(range(len(query_vocab)), query_vocab))
max_decoded_sentence_length = 20

In [ ]:
def decode_sequence_03(input_sentence):
  tokenized_input_sentence = question_vectorization([input_sentence])
  decoded_sentence = "[start]"
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = query_vectorization([decoded_sentence])
    next_token_predictions = seq2seq_03.predict([tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(next_token_predictions[0, i, :])

    sampled_token = query_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence

In [ ]:
actual, predicted = list(), list()
question_texts = [pair[0] for pair in test_pairs]
query_texts = [pair[1] for pair in test_pairs]
for i in range(len(question_texts)):
  input_sentence = question_texts[i]
  output_sentence = query_texts[i]
  y_pred= decode_sequence_03(input_sentence)
  actual.append(output_sentence)
  predicted.append(y_pred)
    
# calcuate BLEU score
print("BLEU-1 Score on Test Dataset: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))

BLEU-1 Score on Test Dataset: 0.145675


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
test_questions_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
  input_sentence = random.choice(test_questions_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence_03(input_sentence))

-
Show the album names and ids for albums that contain tracks with unit price bigger than 1.
[start] select distinct from from where where where ( ( ( select ( select ( select ( select ( select (
-
What are the names and trade names of the medcines that are FDA approved?
[start] select distinct from from where where where ( select ( select ( select ( select ( select ( select (
-
What are the names of enzymes who does not produce 'Heme'?
[start] select distinct from from where where where ( select ( select ( select ( select ( select ( as (
-
Return the type of transaction with the highest total amount.
[start] select distinct from from where where where ( select ( select ( select ( select ( select ( as (
-
What is the total and minimum enrollment of all schools?
[start] select distinct from from where where where ( select ( select ( select ( select ( select ( as (


In [ ]:
actual, predicted = list(), list()
question_texts = [pair[0] for pair in val_pairs]
query_texts = [pair[1] for pair in val_pairs]
for i in range(len(question_texts)):
  input_sentence = question_texts[i]
  output_sentence = query_texts[i]
  y_pred= decode_sequence_03(input_sentence)
  actual.append(output_sentence)
  predicted.append(y_pred)
    
# calcuate BLEU score
print("BLEU-1 Score on Validation Dataset: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))

# Model 4: Three Layer Bidirectional LSTM

In [ ]:
source = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(layers.LSTM(latent_dim), merge_mode="sum")(x)
out_encoder1, state_h_forward1, state_c_forward1, state_h_backward1, state_c_backward1 = layers.Bidirectional(layers.LSTM(latent_dim, return_sequences=True, return_state=True), merge_mode="sum")(x)
out_encoder2, state_h_forward2, state_c_forward2, state_h_backward2, state_c_backward2 = layers.Bidirectional(layers.LSTM(latent_dim, return_sequences=True, return_state=True), merge_mode="sum")(out_encoder1)
out_encoder3, state_h_forward3, state_c_forward3, state_h_backward3, state_c_backward3 = layers.Bidirectional(layers.LSTM(latent_dim, return_sequences=True, return_state=True), merge_mode="sum")(out_encoder2)

In [ ]:
past_target = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_LSTM = layers.LSTM(latent_dim, return_sequences=True)

encoder_state=[state_h_forward3, state_c_forward3]

x = decoder_LSTM(x, initial_state=encoder_state)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)

In [ ]:
seq2seq_04 = keras.Model([source, past_target], target_next_step)

In [ ]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=1),
             ModelCheckpoint("seq2seq_04.keras", save_best_only=True, monitor="val_accuracy", mode='max')]

In [ ]:
seq2seq_04.compile(
optimizer="rmsprop",
loss="sparse_categorical_crossentropy",
metrics=["accuracy"])

seq2seq_04.fit(train_ds, epochs=30, validation_data=val_ds, callbacks=callbacks)

In [ ]:
query_vocab = query_vectorization.get_vocabulary()
query_index_lookup = dict(zip(range(len(query_vocab)), query_vocab))
max_decoded_sentence_length = 20

In [ ]:
def decode_sequence_04(input_sentence):
  tokenized_input_sentence = question_vectorization([input_sentence])
  decoded_sentence = "[start]"
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = query_vectorization([decoded_sentence])
    next_token_predictions = seq2seq_04.predict([tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(next_token_predictions[0, i, :])

    sampled_token = query_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence

In [ ]:
actual, predicted = list(), list()
question_texts = [pair[0] for pair in test_pairs]
query_texts = [pair[1] for pair in test_pairs]
for i in range(len(question_texts)):
  input_sentence = question_texts[i]
  output_sentence = query_texts[i]
  y_pred= decode_sequence_04(input_sentence)
  actual.append(output_sentence)
  predicted.append(y_pred)
    
# calcuate BLEU score
print("BLEU-1 Score on Test Dataset: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))

In [ ]:
test_questions_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
  input_sentence = random.choice(test_questions_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence_04(input_sentence))

In [ ]:
actual, predicted = list(), list()
question_texts = [pair[0] for pair in val_pairs]
query_texts = [pair[1] for pair in val_pairs]
for i in range(len(question_texts)):
  input_sentence = question_texts[i]
  output_sentence = query_texts[i]
  y_pred= decode_sequence_04(input_sentence)
  actual.append(output_sentence)
  predicted.append(y_pred)
    
# calcuate BLEU score
print("BLEU-1 Score on Validation Dataset: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))